In [ ]:
import json

with open("config.json", "r") as f:
    config_argument = json.load(f)

model_predict_all_result_path_txt = config_argument["model_predict_all_result_path_txt"]

whisper_timestamp_word_level_path_json = config_argument["whisper_timestamp_word_level_path_json"]
whisper_timestamp_char_level_path_json = config_argument["whisper_timestamp_char_level_path_json"]

word_level_timestamp_task2_NER_result_path_txt = config_argument["word_level_timestamp_task2_NER_result_path_txt"]
char_level_timestamp_task2_NER_result_path_txt = config_argument["char_level_timestamp_task2_NER_result_path_txt"]


print( "model_predict_all_result_path_txt: ", model_predict_all_result_path_txt )
print( "whisper_timestamp_word_level_path_json: ", whisper_timestamp_word_level_path_json )
print( "whisper_timestamp_char_level_path_json: ", whisper_timestamp_char_level_path_json )
print( "char_level_timestamp_task2_NER_result_path_txt: ", char_level_timestamp_task2_NER_result_path_txt )
print( "word_level_timestamp_task2_NER_result_path_txt: ", word_level_timestamp_task2_NER_result_path_txt )


model_predict_all_result_path:  ./checkpoints/NER_model
whisper_timestamp_word_level_path:  ./checkpoints/NER_model
whisper_timestamp_char_level_path:  ./checkpoints/NER_model
char_level_timestamp_task2_NER_result_path:  
word_level_timestamp_task2_NER_result_path:  


word_level

In [ ]:
def align_ner_with_whisper(ner_entities, whisper_words):
    results = []

    for ent in ner_entities:
        ent_start = ent["start"]
        ent_end = ent["end"]
        name = ent["name"]

        # print(ent_start, ent_end)
        matched_words = []

        for word in whisper_words[str(name)]:
            w_start = word["char_start"]
            w_end = word["char_end"]

            # if int(w_start) <= int(ent_start) <= int(w_end):   # 方法1
            #     matched_words.append(word)

            #檢查是否重疊（部分包含）
            if not (int(w_end) <= int(ent_start) or int(w_start) >= int(ent_end)): # 方法2
                matched_words.append(word)






        print(matched_words)


        if matched_words:
            ts_start = matched_words[0]['start_time']
            ts_end = matched_words[-1]['end_time']
            results.append({
                "entity": ent["word"],
                "start_time": ts_start,
                "end_time": ts_end
            })
        else:
            results.append({
                "entity": ent["word"],
                "start_time": None,
                "end_time": None
            })

    return results

In [ ]:
import json

with open( whisper_timestamp_word_level_path_json, "r" ) as f:
    wisper_word = json.load(f)

with open( model_predict_all_result_path_txt, "r" ) as f:
  data = f.readlines()


pre_dict = {}
pre_list = []

for line in data :

  line = line.strip()
  line_split = line.split("\t")


  pre_dict["name"] = line_split[0]
  pre_dict["type"] = line_split[1]
  pre_dict["start"] = line_split[2]
  pre_dict["end"] = line_split[3]
  pre_dict["word"] = line_split[4]

  pre_list.append(pre_dict)
  pre_dict = {}

# print(pre_list)

# print( pre_list[0] )
# print(wisper_word["271.wav"])

# print(wisper_word)

results = align_ner_with_whisper(pre_list, wisper_word)
# print(results)

In [ ]:
with open( word_level_timestamp_task2_NER_result_path_txt , "w") as f:
    for index, r in enumerate(pre_list):
        start_time = float(results[index]['start_time'])
        end_time = float(results[index]['end_time'])
        f.write(f"{r['name']}\t{r['type']}\t{start_time:.3f}\t{end_time:.3f}\t{results[index]['entity']}\n")


char_level

In [ ]:
def align_char_level_ner_with_whisper(ner_entity, whisper_chars):

    result = []

    for ner in ner_entity:

      name = ner["name"]

      ent_start = int(ner["start"])
      ent_end = int(ner["end"])

      # print(ent_start, ent_end)



      matched_chars = [
          c for c in whisper_chars[name]
          if ent_start <= c["index"] < ent_end
      ]

      if matched_chars:
          ts_start = matched_chars[0]["start"]
          ts_end = matched_chars[-1]["end"]
          result.append( {
              "entity": ner["word"],
              "start_time": ts_start,
              "end_time": ts_end
          } )
      else:
          result.append( {
              "entity": ner["word"],
              "start_time": None,
              "end_time": None
          } )

    return result


In [ ]:
import json

with open( whisper_timestamp_char_level_path_json, "r") as f:
    wisper_word = json.load(f)

with open( model_predict_all_result_path_txt, "r") as f:
  data = f.readlines()


pre_dict = {}
pre_list = []

for line in data :

  line = line.strip()
  line_split = line.split("\t")


  pre_dict["name"] = line_split[0]
  pre_dict["type"] = line_split[1]
  pre_dict["start"] = line_split[2]
  pre_dict["end"] = line_split[3]
  pre_dict["word"] = line_split[4]

  pre_list.append(pre_dict)
  pre_dict = {}

# print(pre_list)

# print( pre_list[0] )
# print(wisper_word["271"])


result = align_char_level_ner_with_whisper(pre_list, wisper_word)

print( result )

In [ ]:
with open( char_level_timestamp_task2_NER_result_path_txt, "w") as f:

  for index, r in enumerate(pre_list):
    f.write(f"{r['name']}\t{r['type']}\t{result[index]['start_time']}\t{result[index]['end_time']}\t{result[index]['entity']}\n")